In [19]:
# Imports
import os
import numpy as np
import nest_asyncio
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from residual import ResidualBlock 
import session_info
import fastapi

'nest_asyncio' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# Load database
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
# Create a path for test images 
image_test_path = './images_test'
if not os.path.exists(image_test_path):
    os.mkdir(image_test_path)

def arraytoimage(x,y):
    ''' Converts numpy array to image'''
    len_x = x.shape[0]
    for i in range(len_x):
        image = Image.fromarray(x[i])
        name = str(y[i])
        image.save(f'{image_test_path }/{name}.png')



In [ ]:
arraytoimage(x_test,y_test)

In [ ]:
# Scale test and training set to range from 0 to 1
x_train, x_test = x_train/255.0, x_test/255.0

In [ ]:
# Model definition

model = keras.Sequential()
model.add(keras.layers.Input(shape=(28,28,1)))

# Keep the first stack of layers of ResNet with a little change in the
# number of strides, set to 1 instead of 2 (due to the size of images)
model.add(keras.layers.Conv2D(64, kernel_size=7, strides=1, name='label_xf'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=1, padding="same"))

# Number of filters in the first Con2D layer
pre_filter=64

# Builds the residual blocks using ResidualBlock class
# defined in the residual.py file
for filter in [64]*3  + [128]:
   strides=1 if filter==pre_filter else 2
   model.add(ResidualBlock(filters=filter, strides=strides))
   pre_filter=filter

# Uses a glabal average layer and then flattens its output
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())

# Add a dense layer and tuner its number of units
model.add(keras.layers.Dense(units=28, activation='relu'))

# Add the output layer
model.add(keras.layers.Dense(10, activation='softmax'))

# Adam optimizer and compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer,
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(x=x_train,y=y_train, batch_size=32, epochs=5, validation_split=0.2, steps_per_epoch=1500)

In [ ]:
# Evaluate the model
model.evaluate(x_test,y_test)

In [ ]:
# Save the model
model.save('model.keras')

### FastAPI app

In [ ]:
app_path = 'main.py'

In [ ]:
!pip install numpy 

In [15]:
#%%writefile {app_path}

# Imports
import nest_asyncio
import io
import cv2
import uvicorn
import cvlib as cv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from residual import ResidualBlock 
from fastapi import FastAPI, UploadFile, File, HTTPException

app= FastAPI(title='Handwriten_digit_Identifier')

@app.get("/")
def homepage():
    return "Welcome to the home page!!! To test the app, go to  http://127.0.0.2:8080/docs"

@app.post("/predict/")
async def prediction(file: UploadFile = File(...)):

    filename = file.filename
    fileExtention = filename.split('.')[-1] in ('jpeg','jpg','png')
    if not fileExtention:
        raise HTTPException(status_code=415, detail="Unsupported file provided.")

    # Read the image as a stream of bytes
    image_stream = io.BytesIO(file.file.read())
    
    # Start the stream from the beginning (position zero)
    image_stream.seek(0)
    
    # Write the stream of bytes into a numpy array
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    
    # Decode the numpy array as an image
    image = cv2.imdecode(file_bytes, cv2.IMREAD_GRAYSCALE)

    # Scale the image and reshape it
    image_0_1 = (image/255).reshape(-1,28,28,1)

    # Load the model
    model = keras.models.load_model("model.keras")
    
    # Prediction 
    y_pred = np.argmax(model.predict(image_0_1))
    conf_prob = np.max(model.predict(image_0_1))

    message= f"The handwriten digit you provided is predicted to be {y_pred} with a confidence of:{conf_prob}"
    
    return {'message': message}

In [ ]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Define a localhost
host = "127.0.0.2"

# Spin up the server!    
uvicorn.run(app, host=host, port=8090)